# ***Project***

---
Create a simple assistant that uses any LLM and should be pydantic, when we ask about any product it should give you two information product Name, product details tentative price in USD(integer). Use chat prompt template and chain system.

---

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing import Annotated, List
from pydantic import BaseModel, Field
load_dotenv()

True

In [21]:
groq_api_key  = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]="Simple Chatbot"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

model = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct", groq_api_key=groq_api_key)

## Product output schema

In [22]:
class ProductDetails(BaseModel):
    """This is product info Schema."""
    
    product_name: Annotated[str, Field(description="This is product name.")]
    product_details: Annotated[List[str], Field(description="List of product details like Ram, Rom, processor, Camera, battery, Screen size.")]
    product_price: Annotated[int, Field(description="Product prince in USD(integer format price).")]
    
parser = PydanticOutputParser(pydantic_object=ProductDetails)

## System messages and ChatPromptTemplate

In [23]:
system_messages = """
    Think you are a tech expert. Your task is user will give a phone name and you will be provide a Structured output based on the {format_instruction} schema.
    1. product name which user will be give you.
    2. product details like Ram, Rom, processor, Camera, battery, Screen size and other which is necessary.
    3. product tentative price in USD format. must in USD integer format.
"""

In [24]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_messages),
        ("human", "{input}")
    ]
).partial(format_instruction=parser.get_format_instructions())

In [25]:
chain = chat_prompt | model | parser

In [33]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
 +--------------------+  
 | ChatPromptTemplate |  
 +--------------------+  
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
   +----------------+    
   | ProductDetails |    
   +----------------+    


In [27]:
response = chain.invoke(
    {
        "input": "Sumsang s24 ultra"
    }
)
response

ProductDetails(product_name='Samsung S24 Ultra', product_details=['12GB/16GB RAM', '128GB/256GB/512GB/1TB ROM', 'Qualcomm Snapdragon 8 Gen 3', '50MP Front Camera, 200MP Rear Camera', '5000mAh battery', '6.8 inches Dynamic AMOLED 2X Screen', '120Hz Refresh Rate', 'IP67 Water and Dust Resistance', 'Ultrasonic Fingerprint Sensor', 'S-Pen Stylus Support'], product_price=1199)

In [28]:
response.model_dump()

{'product_name': 'Samsung S24 Ultra',
 'product_details': ['12GB/16GB RAM',
  '128GB/256GB/512GB/1TB ROM',
  'Qualcomm Snapdragon 8 Gen 3',
  '50MP Front Camera, 200MP Rear Camera',
  '5000mAh battery',
  '6.8 inches Dynamic AMOLED 2X Screen',
  '120Hz Refresh Rate',
  'IP67 Water and Dust Resistance',
  'Ultrasonic Fingerprint Sensor',
  'S-Pen Stylus Support'],
 'product_price': 1199}

In [29]:
response.product_details

['12GB/16GB RAM',
 '128GB/256GB/512GB/1TB ROM',
 'Qualcomm Snapdragon 8 Gen 3',
 '50MP Front Camera, 200MP Rear Camera',
 '5000mAh battery',
 '6.8 inches Dynamic AMOLED 2X Screen',
 '120Hz Refresh Rate',
 'IP67 Water and Dust Resistance',
 'Ultrasonic Fingerprint Sensor',
 'S-Pen Stylus Support']

In [30]:
response.product_price

1199